 相比V1，将相关过程定义成函数。
 
variousCity指武汉、重庆、成都、南京、昆明

variousClass 指根据指数和JRC，获取陆地和永久性水体的范围，未考虑季节性水体，在此类区域上选择样本

In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
wuhan = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
chongqing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing').first()).geometry()
chengdu = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chengdu').first()).geometry()
nanjing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/nanjing').first()).geometry()
kunming = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/kunming').first()).geometry()

city = ee.FeatureCollection([wuhan,chongqing,chengdu,nanjing,kunming])
Map.addLayer(city, {}, "city")
Map.centerObject(city,6)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# 计算指数
def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi)

# # 计算影像裁剪后的面积
# def area_after_clip(image):
#     image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
#     areas = image_area.reduceRegion(**{
#         'reducer': ee.Reducer.sum(),
#         'geometry': image.geometry(),
#         'scale': 30,
#         'maxPixels': 1e14
#     })
#     return image.set({'area': areas.get('B2')})

In [5]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi']
# 分类标签
label = 'waterclass'

# 武汉

In [6]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

basemap = ee.Image.constant(0).clip(wuhan).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(wuhan).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC frequency
basemap = ee.Image.constant(0).clip(wuhan).rename('waterclass')
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# 交集数据部分
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landWaterExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)

# 计算影像裁剪后的面积
def wuhan_area_after_clip(image):
    image_area = image.clip(wuhan).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})
sample_image = landsat_images.map(wuhan_area_after_clip).filter(ee.Filter.gt('area',170))

def wuhanimageSample(image):
    permanent_image = image.clip(wuhan).updateMask(permanentWaterExtent)
    permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
    land_image = image.clip(wuhan).updateMask(landWaterExtent)
    land = landWaterExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
    points_permanentwater = permanent.sample(**{
        'region': wuhan,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_land = land.sample(**{
        'region': wuhan,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    all_points = points_permanentwater.merge(points_land)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points
wuhan_pointsCollection = sample_image.map(wuhanimageSample).flatten()
print(sample_image.size().getInfo())
print(wuhan_pointsCollection.size().getInfo())

40
689


# 成都

In [7]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chengdu) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

basemap = ee.Image.constant(0).clip(chengdu).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(chengdu).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC frequency
basemap = ee.Image.constant(0).clip(chengdu).rename('waterclass')
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(chengdu).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# 交集数据部分
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landWaterExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)

# 计算影像裁剪后的面积
def chengdu_area_after_clip(image):
    image_area = image.clip(chengdu).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})
sample_image = landsat_images.map(chengdu_area_after_clip).filter(ee.Filter.gt('area',170))

def chengduimageSample(image):
    permanent_image = image.clip(chengdu).updateMask(permanentWaterExtent)
    permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
    land_image = image.clip(chengdu).updateMask(landWaterExtent)
    land = landWaterExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
    points_permanentwater = permanent.sample(**{
        'region': chengdu,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_land = land.sample(**{
        'region': chengdu,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    all_points = points_permanentwater.merge(points_land)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points
chengdu_pointsCollection = sample_image.map(chengduimageSample).flatten()
print(sample_image.size().getInfo())
print(chengdu_pointsCollection.size().getInfo())

37
431


# 南京

In [ ]:
roi = nanjing

sample_image = landsatCollection(roi).map(area_after_clip).filter(ee.Filter.gt('area',658))
nanjing_pointsCollection = sample_image.map(imageSample).flatten()
# print(sample_image.size().getInfo())
# print(nanjing_pointsCollection.size().getInfo())

# 昆明

In [ ]:
roi = kunming

sample_image = landsatCollection(roi).map(area_after_clip).filter(ee.Filter.gt('area',2012))
kunming_pointsCollection = sample_image.map(imageSample).flatten()
# print(sample_image.size().getInfo())
# print(kunming_pointsCollection.size().getInfo())

# 样本集，训练，分类

In [8]:
pointsCollection = wuhan_pointsCollection.merge(chengdu_pointsCollection)

trainedClassifier = ee.Classifier.smileRandomForest(20).train(pointsCollection,label,bands)

In [ ]:
# city = ee.FeatureCollection([wuhan,chongqing,chengdu,nanjing,kunming])
# print(landsatCollection(city).size().getInfo())

In [ ]:
# waterpixel_count = landsatCollection(city).map(training).sum()
# validPixel_count = landsatCollection(city).count().select('B3').clip(roi).rename('count')
# waterfrequency = waterpixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
# Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency")

# permanentwater = waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
# Map.addLayer(permanentwater,{'palette':['blue']},"permanent water")

In [10]:
basemap = ee.Image.constant(0).clip(wuhan).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)
def training(image):
    return image.clip(wuhan).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
waterpixel_count = landsat_images.map(training).sum()
validPixel_count = landsat_images.count().select('B3').clip(wuhan).rename('count')
wuhan_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
Map.addLayer(wuhan_waterfrequency,{'palette':['white','green'],'min':0,'max':1},"wuhan water frequency")

In [11]:
wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(wuhan_permanentwater,{'palette':['blue']},"wuhan permanent water")

In [12]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chengdu) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)
def training(image):
    return image.clip(chengdu).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
waterpixel_count = landsat_images.map(training).sum()
validPixel_count = landsat_images.count().select('B3').clip(chengdu).rename('count')
chengdu_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
Map.addLayer(chengdu_waterfrequency,{'palette':['white','green'],'min':0,'max':1},"chengdu water frequency")

In [13]:
chengdu_permanentwater = chengdu_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(chengdu_permanentwater,{'palette':['blue']},"chengdu permanent water")

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(nanjing) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)
def training(image):
    return image.clip(nanjing).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
waterpixel_count = landsat_images.map(training).sum()
validPixel_count = landsat_images.count().select('B3').clip(nanjing).rename('count')
nanjing_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
Map.addLayer(nanjing_waterfrequency,{'palette':['white','green'],'min':0,'max':1},"nanjing water frequency")

In [ ]:
nanjing_permanentwater = nanjing_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(nanjing_permanentwater,{'palette':['blue']},"nanjing permanent water")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
# use cartoee to get a map
ax = cartoee.get_map(wuhan_waterfrequency, region=region, vis_params=vis,cmap = cmap)
# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(wuhan)', fontsize=20)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['0000ff']}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(wuhan_permanentwater, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(wuhan)', fontsize=20)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.95, 30.0, 104.95, 31.5] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
# use cartoee to get a map
ax = cartoee.get_map(chengdu_waterfrequency, region=region, vis_params=vis,cmap = cmap)
# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(chengdu)', fontsize=20)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.95, 30.0, 104.95, 31.5]
vis = {'palette':['0000ff']}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(chengdu_permanentwater, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(chengdu)', fontsize=20)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [118.3, 31.16, 119.3, 32.7] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
# use cartoee to get a map
ax = cartoee.get_map(nanjing_waterfrequency, region=region, vis_params=vis,cmap = cmap)
# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(nanjing)', fontsize=20)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [118.3, 31.16, 119.3, 32.7] 
vis = {'palette':['0000ff']}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(nanjing_permanentwater, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(nanjing)', fontsize=20)

# 附录

In [ ]:
points_collection = sample_image.map(imageSample).flatten()
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
def training(image):
    return image.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')

waterpixel_count = landsat_images.map(training).sum()
validPixel_count = landsat_images.count().select('B3').clip(roi).rename('count')
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency")

permanentwater = waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(permanentwater,{'palette':['blue']},"permanent water")

In [ ]:
def waterfrequency(roi):
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    water_layer = landsatCollection(roi).map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
    validPixel_layer = landsatCollection(roi).count().select('B3').clip(roi).rename('count')
    index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
    index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
    index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
    # JRC frequency
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
    # 交集数据部分
    extent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2).rename('permanent')
    landWaterExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2).rename('land')
    return extent.addBands(landWaterExtent)

In [ ]:
def imageSample(image):
    permanentWaterExtent = waterfrequency(roi).select('permanent').rename('waterclass')
    landExtent = waterfrequency(roi).select('land').rename('waterclass')
    permanent_image = image.clip(roi).updateMask(permanentWaterExtent)
    permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
    land_image = image.clip(roi).updateMask(landExtent)
    land = landExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
    points_permanentwater = permanent.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_land = land.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    all_points = points_permanentwater.merge(points_land)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points